In [7]:
import os
import pandas as pd
import xlwings as xw

def one_hot_to_integer(one_hot_sequence):
    """Convert a one-hot encoded sequence to an integer based on the position of the '1'."""
    for idx, val in enumerate(one_hot_sequence):
        if val == 1:
            return idx + 1  # Add 1 to the index to shift up by one
    return 0  # Return 0 if no '1' is found

def f3_split(df):
    split_columns = {}
    for i in range(7):
        col_name = f'F3_{chr(ord("a") + i)}'
        split_columns[col_name] = df.iloc[i + 19, 3]
    return split_columns

def extract_id_from_of_sheet(excel_file):
    # Read the 'OF' sheet
    df = pd.read_excel(excel_file, sheet_name='OF', header=None)
    # Extract the ID from cell D4
    id_value = df.iat[3, 3]
    # Convert ID to numeric if possible
    try:
        id_value = pd.to_numeric(id_value)
    except ValueError:
        pass  # ID is not numeric, leave it as is
    return id_value
    
def binary_to_int(binary_sequence):
    """Convert a binary encoded sequence to an integer."""
    # Convert float values to integers (0 or 1)
    binary_sequence = [int(val) for val in binary_sequence if not pd.isna(val)]
    binary_str = ''.join(map(str, binary_sequence))
    return int(binary_str, 2)

def f_65(df, data_cells):
    """Special function for processing f_65."""
    data_row = int(data_cells[1:]) - 1
    data_col = ord(data_cells[0]) - ord('A')
    value = df.iat[data_row, data_col]
    if pd.isna(value):
        return 0  # Return 0 if the cell is empty
    return value  # Return the float value if it is present


#REMOVED OF1(A5), OF12(A73), OF29(A152), OF32(A162),  OF35(A166),  OF36(A167), 
#Removed F11(A72),  F26(A165), F27(A166), F42(A123), F60(A321), F62(A322), f66a (a341)
#removed s3/s6 A40/90
def read_and_save_excel_data(input_files, output_csv):
    # Define the sheets and the cells to extract data from
    sheets = ['OF', 'F','S','Scores']
    data_to_extract = {
        'OF': [
            ('A10', 'D11:D16', 'binary'),
            ('A18', 'D19:D24', 'binary'),
            ('A25', 'D26:D32', 'binary'),
            ('A33', 'D34:D40', 'binary'),
            ('A42', 'D42', 'float'),
            ('A44', 'D44', 'float'),
            ('A46', 'D47:D51', 'binary'),
            ('A53', 'D54:D58', 'binary'),
            ('A60', 'D61:D63', 'binary'),
            ('A65', 'D66:D71', 'binary'),
            ('A74', 'D75:D81', 'binary'),
            ('A82', 'D83:D88', 'binary'),
            ('A89', 'D90:D95', 'binary'),
            ('A97', 'D98:D102', 'binary'),
            ('A103', 'D104:D107', 'binary'),
            ('A109', 'D109', 'float'),
            ('A110', 'D110', 'float'),
            ('A111', 'D112:D115', 'binary'),
            ('A116', 'D117:D120', 'binary'),
            ('A121', 'D122:D125', 'binary'),
            ('A127', 'D128:D130', 'binary'),
            ('A131', 'D132:D134', 'binary'),
            ('A135', 'D136:D138', 'binary'),
            ('A139', 'D140:D145', 'binary'),
            ('A147', 'D147', 'float'),
            ('A148', 'D149:D150', 'binary'),
            ('A160', 'D160', 'float'),
            ('A161', 'D161', 'float'),
            ('A164', 'D164', 'float'),
            ('A165', 'D165', 'float'),
            ('A168', 'D168', 'float'),
            ('A170', 'D171:D174', 'binary')
        ],
        'F': [
            ('A5', 'D7:D8,D10:D11', 'binary'),
            ('A13', 'D14:D17', 'binary'),
            ('A19', 'D20:D26', 'f3_split'),
            ('A28', 'D29:D31', 'binary'),
            ('A33', 'D34:D41', 'binary_to_int'),
            ('A42', 'D44:D45,D47:D48', 'binary'),
            ('A49', 'D50:D54', 'binary'),
            ('A56', 'D57:D61', 'binary'),
            ('A63', 'D64:D67', 'binary'),
            ('A69', 'D70:D71', 'binary'),
            ('A79', 'D80:D83', 'binary'),
            ('A84', 'D85:D89', 'binary'),
            ('A91', 'D92:D96', 'binary'),
            ('A98', 'D99:D101', 'binary'),
            ('A103', 'D104:D105', 'binary'),
            ('A107', 'D108:D112', 'binary'),
            ('A114', 'D115:D117', 'binary'),
            ('A118', 'D119:D123', 'binary'),
            ('A124', 'D125:D127', 'binary'),
            ('A128', 'D129:D133', 'binary'),
            ('A135', 'D136:D140', 'binary'),
            ('A142', 'D143:D147', 'binary'),
            ('A149', 'D150:D155', 'binary'),
            ('A158', 'D159:D163', 'binary'),
            ('A168', 'D169:D173', 'binary'),
            ('A175', 'D176:D180', 'binary'),
            ('A182', 'D183:D185', 'binary'),
            ('A187', 'D188:D192', 'binary'),
            ('A194', 'D194', 'float'),
            ('A195', 'D196:D201', 'binary'),
            ('A202', 'D203:D208', 'binary'),
            ('A210', 'D211:D215', 'binary'),
            ('A217', 'D218:D220', 'binary'),
            ('A222', 'D222', 'float'),
            ('A223', 'D223', 'float'),
            ('A224', 'D224', 'float'),
            ('A225', 'D226:D228', 'binary'),
            ('A230', 'D230', 'float'),
            ('A232', 'D233:D237', 'binary'),
            ('A239', 'D240:D242', 'binary'),
            ('A244', 'D245:D249', 'binary'),
            ('A250', 'D250', 'float'),
            ('A251', 'D252:D254', 'binary'),
            ('A255', 'D256:D258', 'binary'),
            ('A260', 'D261:D264', 'binary'),
            ('A266', 'D267:D271', 'binary'),
            ('A272', 'D273:D274', 'binary'),
            ('A275', 'D276:D279', 'binary'),
            ('A280', 'D281:D285', 'binary'),
            ('A286', 'D287:D292', 'binary'),
            ('A294', 'D295:D298', 'binary'),
            ('A300', 'D301:D303', 'binary'),
            ('A304', 'D305:D307', 'binary'),
            ('A308', 'D309:D314', 'binary'),
            ('A316', 'D317:D320', 'binary'),
            ('A323', 'D324:D330', 'binary'),
            ('A332', 'D332', 'float'),
            ('A333', 'D334:D336', 'binary'),
            ('A337', 'D338', 'float'),
            ('A343', 'D343', 'float'),
            ('A344', 'D344', 'float'),
            ('A345', 'D345', 'float')
        ],
        'S': [
            ('A5', 'F26', 'float'),
            ('A27', 'F39', 'float'),
            ('A53', 'F70', 'float'),
            ('A71', 'F88', 'float'),
        ],

        'Scores': [
            ('B8', 'C8', 'xlwings'),
            ('B8', 'E8', 'xlwings'),  # Using xlwings to get value
            ('B9', 'C9', 'xlwings'),
            ('B9', 'E9', 'xlwings'),
            ('B10', 'C10', 'xlwings'),
            ('B10', 'E10', 'xlwings'),
            ('B11', 'C11', 'xlwings'),
            ('B11', 'E11', 'xlwings'),
            ('B12', 'C12', 'xlwings'),
            ('B12', 'E12', 'xlwings')
        ]
    }

    # Mapping cell references to column names
    column_names = {
        'C8': 'WS',
        'E8': 'WS_Benefit',
        'C9': 'SFST',
        'E9': 'SFST_Benefit',
        'C10': 'SR',
        'E10': 'SR_Benefit',
        'C11': 'PR',
        'E11': 'PR_Benefit',
        'C12': 'NR',
        'E12': 'NR_Benefit'
    }

    # Initialize an empty list to store DataFrames for each file
    df_list = []
    app = xw.App(visible=False)
    # Iterate over the input Excel files
    hhh=0
    for excel_file in input_files:
        print(hhh)
        hhh+=1
        # Extract the ID from the 'OF' sheet
        file_id = extract_id_from_of_sheet(excel_file)

        # Initialize a dictionary to store extracted data for this file
        extracted_data = {'id': file_id}

        # Iterate over the sheets and extract data
        for sheet in sheets:
            df = pd.read_excel(excel_file, sheet_name=sheet, header=None)
            try:
                df.book.set_calculation_mode('automatic')  # Set calculation mode to automatic
                df.book.calculate()  # Recalculate the workbook
            except Exception as e:
                print(f"Error occurred while recalculating workbook: {e}")

            for col_name_cell, data_cells, processing_method in data_to_extract[sheet]:
                # Extract the column name
                col_name_row = int(col_name_cell[1:]) - 1
                col_name_col = ord(col_name_cell[0]) - ord('A')
                col_name = df.iat[col_name_row, col_name_col]

                if pd.isna(col_name):
                    print(f"Warning: Column name at {col_name_cell} is NaN.")
                    continue

                if processing_method == 'binary':
                    # Extract and concatenate the cell ranges
                    data_ranges = [range_str.split(':') for range_str in data_cells.split(',')]
                    concatenated_data = []
                    for range_start, range_end in data_ranges:
                        start_row = int(range_start[1:]) - 1
                        end_row = int(range_end[1:]) - 1
                        start_col = ord(range_start[0]) - ord('A')
                        end_col = ord(range_end[0]) - ord('A')
                        concatenated_data.extend(df.iloc[start_row:end_row + 1, start_col:end_col + 1].values.flatten())

                    # Convert the concatenated data to one-hot encoded sequence
                    encoded_value = one_hot_to_integer(concatenated_data)
                    extracted_data[col_name] = encoded_value
                elif processing_method == 'float':
                    # Directly extract the float value
                    data_row = int(data_cells[1:]) - 1
                    data_col = ord(data_cells[0]) - ord('A')
                    encoded_value = df.iat[data_row, data_col]
                    extracted_data[col_name] = encoded_value
                elif processing_method == 'f3_split':
                    # Use custom processing method
                    extracted_data.update(f3_split(df))
                elif processing_method == 'binary_to_int':
                    # Extract and concatenate the cell ranges
                    data_ranges = [range_str.split(':') for range_str in data_cells.split(',')]
                    concatenated_data = []
                    for range_start, range_end in data_ranges:
                        start_row = int(range_start[1:]) - 1
                        end_row = int(range_end[1:]) - 1
                        start_col = ord(range_start[0]) - ord('A')
                        end_col = ord(range_end[0]) - ord('A')
                        concatenated_data.extend(df.iloc[start_row:end_row + 1, start_col:end_col + 1].values.flatten())

                    # Convert the concatenated data to a binary integer
                    encoded_value = binary_to_int(concatenated_data)
                    extracted_data[col_name] = encoded_value
                elif processing_method == 'xlwings':
                    # Extract the value using xlwings
                    col_name = column_names[data_cells]
                    value = extract_value_with_xlwings(excel_file, data_cells,app)
                    extracted_data[col_name] = value
                else:
                    print(f"Warning: Unknown processing method '{processing_method}' for column '{col_name}'. Skipping.")

        # Append the extracted data for this file to the list of DataFrames
        df_list.append(pd.DataFrame([extracted_data]))

    # Concatenate all DataFrames into a single DataFrame
    extracted_df = pd.concat(df_list, ignore_index=True)

    # Reorder columns so 'id' column is first
    extracted_df = extracted_df[['id'] + [col for col in extracted_df.columns if col != 'id']]

    # Write the DataFrame to a CSV file
    extracted_df.to_csv(output_csv, index=False)

def extract_value_with_xlwings(excel_file, cell,app):
    """Extract value from specified cell using xlwings."""
    
    wb = app.books.open(excel_file)
    value = wb.sheets['Scores'].range(cell).value
    # Round the value to 3 digits
    rounded_value = round(value, 3)
    wb.close()
    return rounded_value

# Specify the input folder containing Excel files and output CSV file
input_folder = 'wespacs'
output_csv_file = '1_output_reg_normalized.csv'

# Get a list of all Excel files in the input folder
input_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.xlsx')]

# Read data from Excel files, encode, and save to CSV
read_and_save_excel_data(input_files, output_csv_file)


0
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
1
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
2
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recalculating workbook: 'DataFrame' object has no attribute 'book'
Error occurred while recal